In [1]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Func

from model import GPTLanguageModel, GPTConfig, Block, LayerNorm
config = GPTConfig()

block_size = 8
batch_size = 4
vocab_size = 50304
n_embd = 6
n_head = 2
dropout = 0.1
device_type = 'cpu'
device = 'cpu'

bin_dir = os.path.join(os.path.join(os.getcwd(), "data"), "bin")

train_data = np.memmap(os.path.join(bin_dir, "train.bin"), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(bin_dir, "val.bin"), dtype=np.uint16, mode='r')

In [ ]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i + block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i + 1:i + block_size + 1]).astype(np.int64)) for i in ix])
    
    if device_type == "cuda":
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    
    return x, y

In [ ]:
wte = nn.Embedding(vocab_size, n_embd)
wpe = nn.Embedding(block_size, n_embd)
attn_layer = nn.Linear(n_embd, 3 * n_embd, bias=True)
proj_layer = nn.Linear(n_embd, n_embd, bias=True)
attn_dropout = nn.Dropout(dropout)
resid_dropout = nn.Dropout(dropout)
dropout_layer = nn.Dropout(0.1)
ln_1 = LayerNorm(n_embd, bias=True)

In [ ]:
X, Y = get_batch("train")
b,t  = X.size()

pos = torch.arange(0, t, dtype=torch.long, device=device)
tok_emb = wte(X)    # tok_emb[0,0,:] - Embedding vector for first token
pos_emb = wpe(pos)
x = dropout_layer(tok_emb + pos_emb)
x = ln_1(x)

Self-Attention

In [ ]:
B,T,C = x.size()

q, k, v = attn_layer(x).split(n_embd, dim=2)
k = k.view(B, T, n_head, C // n_head).transpose(1, 2) # (B, nh, T, hs)
q = q.view(B, T, n_head, C // n_head).transpose(1, 2) # (B, nh, T, hs)
v = v.view(B, T, n_head, C // n_head).transpose(1, 2) # (B, nh, T, hs)

# Causal Self-Attention: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
bias = torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size)
att = att.masked_fill(bias[:,:,:T,:T] == 0, float('-inf'))
att = Func.softmax(att, dim=-1)
att = attn_dropout(att)

y = att @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
y = y.transpose(1, 2).contiguous().view(B, T, C)  # re-assemble all head outputs side by side

# Output projection
y = resid_dropout(proj_layer(y))

In [ ]:
transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),   # Word token embedding
            wpe = nn.Embedding(config.block_size, config.n_embd),   # Positional Encoding Embedding
            dropout = nn.Dropout(config.dropout),
            blocks = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            layer_norm = LayerNorm(config.n_embd, bias=config.bias),
        ))

lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

In [ ]:
X, Y = get_batch("train")

def forward(idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= config.block_size, f"Cannot forward sequence of length {t}, block size is only {config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device)

        # Forward the GPT model
        tok_emb = transformer.wte(idx)
        pos_emb = transformer.wpe(pos)
        x = transformer.dropout(tok_emb + pos_emb)
        for block in transformer.blocks:
            x = block(x)
        x = transformer.layer_norm(x)

        if targets is not None:
            logits = lm_head(x)
            loss = Func.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # Only forward the lm_head on the very last position
            logits = lm_head(x[:, [-1], :])
            loss = None
        
        return logits, loss

In [ ]:
logits, _ = forward(X)
logits = logits[:, -1, :]
logits.size()
probs = Func.softmax(logits, dim=-1)
idx_next = torch.multinomial(probs, num_samples=1)
#idx = torch.cat((idx, idx_next), dim=1)

In [ ]:
#model = GPTLanguageModel(config)

In [ ]:
from trainer import TrainConfig
con = TrainConfig()
con.device

In [ ]:
from trainer import TrainConfig
config = TrainConfig()
config.max_iters = 10

In [ ]:
class Trainer:
    def __init__(self, config):
        self.config = config
    
    def run(self):
        while self.config.iter_num < self.config.max_iters:
            self.config.iter_num += 1
            print(self.config.iter_num)

In [ ]:
trainer = Trainer(config)
trainer.run()

In [2]:
ml = [k for k,v in globals().items() if not k.startswith("_") and isinstance(v, (int, float, bool, str))]
ml

['block_size',
 'batch_size',
 'vocab_size',
 'n_embd',
 'n_head',
 'dropout',
 'device_type',
 'device',
 'bin_dir']